In [15]:
import numpy as np
import matplotlib.pyplot as plt

In [16]:
%ls

accuracy.png       dTb03             Powerspectrum_THANN_burnin.out
binary_data_03/    hyperparam.ipynb  Powerspectrum_THANN_burninprob.out
bk0.3h5/           index             Powerspectrum_THANN_.log
bkkkk0.3.h5        index.png         Powerspectrum_THANN_.out
bk_norm03          loss.png          Powerspectrum_THANN_prob.out
bk_un03            lvac03.png        x_HI03
cosmoHammer.ipynb  nbins03
cov_inv_test       params03


In [17]:
import numpy as np
import matplotlib.pyplot as plt

Bk = np.loadtxt('bk_norm03')
params = np.loadtxt('params03')
nbins = np.loadtxt('nbins03')
Bk =  np.around(Bk,1)
ind= np.loadtxt('index')
index = np.zeros(len(ind),dtype=int)
for i in range(len(ind)):
    index[i] = ind[i]
nbins_test = nbins[index]

Bk_03_test = Bk[index]

Bk_03_train = np.delete(Bk,index,axis=0)

Bk_03_train = Bk_03_train

params_test = params[index]

params_train = np.delete(params,index,axis=0)

params_train = params_train


from tensorflow import keras as ks

model = ks.models.load_model('bkkkk0.3.h5')

#======================================================================================================================#

def Bk_pred(params):
    
    
    Bk_pdct = model.predict(params)
    Bk_pdct = Bk_pdct*10000.
    
    return Bk_pdct



In [18]:
Bk.shape,nbins.shape,params.shape

((1058, 550), (1058, 550), (1058, 3))

array([ 146,  833,  516,  204,  929,  223,  154,   42,   87,   10,  535,
        343,  525,  239,   37,  532,  966, 1043])

In [28]:
def cov(Bk,Nbins):
    a = np.eye(550,550)
    b = Bk/np.sqrt(Nbins)
    b = np.reshape(b,(550,1))
    b = b*a
    cov_inv = np.linalg.inv(b)
    return cov_inv

In [29]:


#======================================================================================================================#

class PScore(object):

    def __init__(self):
        pass                                    #=====================CosmoHammer core module for THANN================#
    def __call__(self, ctx):
        params = ctx.getParams()

        ctx.add('params_pk',params)
        n_ion, R_mfp, NoH = params
        prm = np.array([[n_ion, R_mfp, NoH]])
        prm.reshape(1,3)

        Bk_th = Bk_pred(prm)

        ctx.add("key_data",Bk_th)

    def setup(self):

        print("Powerspectrum core setup is done")

#======================================================================================================================#





In [30]:

#======================================================================================================================#
class PSlikeModule(object):
    def __init__(self):
        pass

    def computeLikelihood(self,ctx):                 #================computed liklihood=========#
        Bk_th = ctx.get("key_data")

        ind = 9
        Bk = np.reshape(Bk_03_test[ind],(1,550))
        diff = np.subtract(Bk,Bk_pred(params))
        diff = diff/Bk
        cov_inv = cov(Bk_03_test[ind],nbins_test[ind])
        

        logl = -0.5*np.dot(np.dot(diff,cov_inv),diff.T)
        #logl = logl[0]
        

        return logl

    def setup(self):

        print("Powerspectrum logLikelihood setup is done")

#======================================================================================================================#


In [32]:
from cosmoHammer import LikelihoodComputationChain
from cosmoHammer.CosmoHammerSampler import CosmoHammerSampler
from cosmoHammer.util import Params
#from Power_core import PScore
#from Power_like import PSlikeModule as slk

#======================================================================================================================#

params = Params(("n_ion",[250.,10.0,510.0,1.0]),("R_mfp", [38.0,5.0,70.0,.5]),("NoH",[750.0,10.0,1510.0,1.0]))
                            #==========The parameter space is defined================================#
#======================================================================================================================#

chain = LikelihoodComputationChain(min = params[:,1] , max = params[:,2])

chain.addCoreModule(PScore())                                       #=========setting up the modules===================#

chain.addLikelihoodModule(PSlikeModule())

chain.setup()
#======================================================================================================================#

sampler = CosmoHammerSampler(
            params= params,
            likelihoodComputationChain=chain,                  #=============mpi sampler===============================#
            filePrefix="Powerspectrum_THANN_",
            walkersRatio=10,
            burninIterations=250,
            sampleIterations=250, threadCount=16)

#======================================================================================================================#

print("start sampling: Here we go with the power of zeus.")
sampler.startSampling()
print("done!")

#======================================================================================================================#


Powerspectrum core setup is done
Powerspectrum logLikelihood setup is done
start sampling: Here we go with the power of zeus.
emcee: Exception while calling your likelihood function:
  params: [249.87747282  38.43803396 748.61753814]
  args: []
  kwargs: {}
  exception:


Traceback (most recent call last):
  File "/home/ht/.local/lib/python3.6/site-packages/emcee/ensemble.py", line 495, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "/home/ht/.local/lib/python3.6/site-packages/cosmoHammer/LikelihoodComputationChain.py", line 103, in __call__
    likelihood = self.computeLikelihoods(ctx)
  File "/home/ht/.local/lib/python3.6/site-packages/cosmoHammer/LikelihoodComputationChain.py", line 144, in computeLikelihoods
    likelihood += self.invokeLikelihoodModule(lModule, ctx)
  File "/home/ht/.local/lib/python3.6/site-packages/cosmoHammer/LikelihoodComputationChain.py", line 152, in invokeLikelihoodModule
    return likelihoodModule.computeLikelihood(ctx)
  File "<ipython-input-30-73e4e858f1d0>", line 11, in computeLikelihood
    diff = np.subtract(Bk,Bk_pred(params))
  File "<ipython-input-27-a512827c3a66>", line 10, in Bk_pred
    Bk_pdct = model.predict(params)
  File "/home/ht/.local/lib/python3.6/site-packages/tensorflow_core/python/

ValueError: Failed to find data adapter that can handle input: <class 'cosmoHammer.util.Params.Params'>, <class 'NoneType'>